# INF581 - Advanced Topics in Artificial Intelligence

<!--<img src="logo.jpg" style="float: left; width: 15%" />-->

[Lab session #04 - Reinforcement Learning I](https://moodle.polytechnique.fr/course/view.php?id=7316)

Jérémie DECOCK

<a href="https://colab.research.google.com/github/jeremiedecock/polytechnique-inf581-2020/blob/master/lab4solution.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<a href="https://mybinder.org/v2/gh/jeremiedecock/polytechnique-inf581-2020/master?filepath=lab4solution.ipynb"><img align="left" src="https://mybinder.org/badge.svg" alt="Open in Binder" title="Open and Execute in Binder"></a>

## Objectives

In this lab session, we implement classic *Dynamic Programming* algorithms:
- Value Iteration
- Policy Iteration

## Imports

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import math
import gym
import numpy as np
import copy
import pandas as pd
import seaborn as sns

In [ ]:
sns.set_context("talk")

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

Load the environment (FrozenLake-v0)

In [ ]:
env = gym.make('FrozenLake-v0')

**Notice**: this environment is *fully observable* thus here the terms (environment) *state* and (agent) *observation* are equivalent.
This is not always the case for example in poker, the agent doesn't know the opponent's cards.

For more information on FrozenLake-V0: https://github.com/openai/gym/wiki/FrozenLake-v0

## Get the environment state space and action space

In [ ]:
states = list(range(env.observation_space.n))
states

In [ ]:
actions = list(range(env.action_space.n))
actions

The following dictionary may be used to understand actions:

In [ ]:
action_labels = {
    0: "Move Left",
    1: "Move Down",
    2: "Move Right",
    3: "Move Up"
}

## Display functions

In [ ]:
def states_display(state_seq, title=None, figsize=(5,5), annot=True, fmt="0.1f", linewidths=.5, square=True, cbar=False, cmap="Reds"):
    size = int(math.sqrt(len(state_seq)))
    state_array = np.array(state_seq)
    state_array = state_array.reshape(size, size)
    
    fig, ax = plt.subplots(figsize=figsize)         # Sample figsize in inches
    sns.heatmap(state_array, annot=annot, fmt=fmt, linewidths=linewidths, square=square, cbar=cbar, cmap=cmap)
    plt.title(title)
    plt.show()

In [ ]:
def transition_display(state, action):
    states_display(transition_array[state,action], title="Transition probabilities for action {} ({}) in state {}".format(action, action_labels[action], state))

In [ ]:
def display_policy(policy):
    actions_src = ["{}={}".format(action, action_labels[action].replace("Move ", "")) for action in actions]
    title = "Policy (" + ", ".join(actions_src) + ")"
    states_display(policy, title=title, fmt="d", cbar=False, cmap="Reds")

## Make the `is_final_array`, `reward_array` and `transition_array`

To implement Dynamic Programming algorithms, you will need the transition probability (or transition function) and the reward function, both defined in `env.P`.

Use `env.P[S][A]` to get the list of reachable states from state S executing action A.

These reachable states are coded in a tuple defined like this: `(probability, next state, reward, is_final_state)`.

In [ ]:
is_final_array = np.full(shape=len(states), fill_value=np.nan, dtype=np.bool)
reward_array = np.full(shape=len(states), fill_value=np.NINF)                # np.NINF = negative infinity
transition_array = np.zeros(shape=(len(states), len(actions), len(states)))

for state in states:
    for action in actions:
        for next_state_tuple in env.P[state][action]:              # env.P[state][action] contains the next states list (a list of tuples)
            transition_probability, next_state, next_state_reward, next_state_is_final = next_state_tuple

            is_final_array[next_state] = next_state_is_final
            reward_array[next_state] = max(reward_array[next_state], next_state_reward)   # workaround: when we already are in state 15, reward is 0 if we stay in state 15 (in practice this never append as the simulation stop when we arrive in state 15 as any other terminal state)
            transition_array[state, action, next_state] += transition_probability

In [ ]:
states_display(states, fmt="d", title="States ID")

In [ ]:
states_display(reward_array, title="Rewards")

In [ ]:
states_display(is_final_array, fmt="d", title="Final states")

In [ ]:
# Print final states ID
is_final_array.nonzero()

Display some transitions:

In [ ]:
transition_display(state=0, action=0)

In [ ]:
transition_display(state=6, action=0)

In [ ]:
transition_display(state=6, action=1)

## Value Iteration

### Define the expected value functions

In [ ]:
def expected_value(state, action, v_array):
    return (transition_array[state, action] * v_array).sum() # compute sum(T(s,a,s').V(s'))

In [ ]:
def expected_values(state, v_array):
    return (transition_array[state] * v_array).sum(axis=1)   # compute sum(T(s,a,s').V(s')) for all actions

### Compute states value

In [ ]:
stop = False

value_function_history = []
delta_history = []

def value_iteration(gamma=0.95, epsilon=0.001, display=False):
    v_array = np.zeros(len(states))   # Initial value function
    stop = False

    while not stop:
        if display:
            states_display(v_array, title="Value function", cbar=True, cmap="Reds")
        else:
            print('.', end="")
        value_function_history.append(v_array)
        new_v_array = copy.deepcopy(v_array)
        
        delta = 0.
        
        for state in states:
            if is_final_array[state]:
                new_v_array[state] = reward_array[state]
            else:
                #assert expected_values(state, v_array).max() == max(  [expected_value(state, action, v_array) for action in actions]  )
                #new_v_array[state] = reward_array[state] + gamma * max(  [expected_value(state, action, v_array) for action in actions]  )
                new_v_array[state] = reward_array[state] + gamma * expected_values(state, v_array).max()
            
            delta = max(abs(new_v_array[state] - v_array[state]), delta)
        
        delta_history.append(delta)
        v_array = new_v_array
        
        if delta < epsilon:
            stop = True
    
    return v_array
        
v_array = value_iteration(display=True)
states_display(v_array, title="Value function", cbar=True, cmap="Reds")

In [ ]:
df_v_hist = pd.DataFrame(value_function_history)
df_v_hist

In [ ]:
df_v_hist.plot()
plt.title("V(s) w.r.t iteration")
plt.ylabel("V(s)")
plt.xlabel("iteration")
plt.legend(loc='upper right');

In [ ]:
plt.plot(delta_history)
plt.yscale("log")
plt.title(r"$\max~\delta$ w.r.t iteration")
plt.ylabel(r"$\max~\delta$")
plt.xlabel("iteration");

### Define the (greedy) policy (Maximum Expected Utility)

In [ ]:
def greedy_policy(state, v_array):
    return expected_values(state, v_array).argmax()

### Display the opimized policy

In [ ]:
policy = [greedy_policy(state, v_array) for state in states]
display_policy(policy)

### Evaluate Value Iteration with Gym (single trial)

In [ ]:
env._max_episode_steps = 1000

In [ ]:
reward_list = []

NUM_EPISODES = 1000

for episode_index in range(NUM_EPISODES):
    state = env.reset()
    done = False
    #t = 0

    while not done:
        action = greedy_policy(state, v_array)
        state, reward, done, info = env.step(action)
        #t += 1

    reward_list.append(reward)
    #print("Episode finished after {} timesteps ; reward = {}".format(t, reward))

print(sum(reward_list) / NUM_EPISODES)            

env.close()

### Evaluate Value Iteration for different $\gamma$ with confidence interval (bootstrap)

In [ ]:
%%time

NUM_EPISODES = 1000

reward_list = []

for gamma in (0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.84, 0.9, 0.95, 0.99):
    v_array = value_iteration(gamma=gamma)
    
    for episode_index in range(NUM_EPISODES):
        state = env.reset()
        done = False

        while not done:
            action = greedy_policy(state, v_array)
            state, reward, done, info = env.step(action)

        reward_list.append({"gamma": gamma, "reward": reward})

env.close()

In [ ]:
df = pd.DataFrame(reward_list)
df.tail()

In [ ]:
# Plot mean reward (with its 95% confidence interval)

sns.relplot(x="gamma", y="reward", kind="line", data=df, height=6, aspect=1.5)
plt.axhline(0.76, color="red", linestyle=":", label="76% success threshold");   # 76% success threshold
plt.legend();

### Display the VI optimal policy w.r.t. $\gamma$

In [ ]:
for gamma in (0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.84, 0.9, 0.95, 0.99):
    print()
    print("=" * 10, "GAMMA = ", gamma, "=" * 10)
    print()
    
    v_array = value_iteration(gamma=gamma)
    
    print()
    print()
    
    policy = [greedy_policy(state, v_array) for state in states]
    display_policy(policy)

## Policy Iteration

### Exploratory code: evaluate a given policy (i.e. compute the corresponding value function)

In [ ]:
policy = np.zeros(len(states), dtype=int)
policy

In [ ]:
A = np.full(shape=(len(states), len(states)), fill_value=np.nan)

for state in states:
    for next_state in states:
        action = policy[state]
        A[state, next_state] = -gamma * transition_array[state, action, next_state]
        
        if state == next_state:
            A[state, next_state] = A[state, next_state] + 1.
    
        #if is_final_array[state]:
        #    A[state, next_state] = reward_array[state]   # <- Singular matrix

In [ ]:
transition_display(state=15, action=0)

In [ ]:
sns.heatmap(A, annot=True, fmt="0.2f", linewidths=.5, cmap="Reds") # , square=True
plt.title(r"Coeficients of the system of Bellman equations for the given policy $\pi$ and for $\gamma$=" + str(gamma));

In [ ]:
v_array = np.dot(np.linalg.inv(A), reward_array)   # x is the value function
states_display(v_array, title="Value function", cbar=True, cmap="Reds")

### Define the (exact) Policy Evaluation function

In [ ]:
def policy_evaluation(policy, gamma):
    A = np.full(shape=(len(states), len(states)), fill_value=np.nan)

    for state in states:
        for next_state in states:
            action = policy[state]
            A[state, next_state] = -gamma * transition_array[state, action, next_state]

            if state == next_state:
                A[state, next_state] = A[state, next_state] + 1.

    x = np.dot(np.linalg.inv(A), reward_array)
    
    return x

In [ ]:
policy = np.ones(len(states), dtype=int)
#policy = np.zeros(len(states), dtype=int)
#policy = np.array([0, 3, 3, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype='int') # Optimal policy found by Value Iteration

display_policy(policy)

v_array = policy_evaluation(policy, gamma)
states_display(v_array, title="Value function", cbar=True)

### Define the Policy Improvement function

In [ ]:
def policy_iteration(gamma, initial_policy=None, policy_evaluation_function=policy_evaluation):
    # Set the initial policy
    if initial_policy is not None:
        policy = initial_policy
    else:
        policy = np.random.randint(low=min(actions), high=max(actions), size=len(states), dtype='int')  # Random initial policy

    stop = False
    while not stop:
        print(policy)

        v_array = policy_evaluation_function(policy, gamma)

        new_policy = np.copy(policy)
        for state in states:
            new_policy[state] = greedy_policy(state, v_array)
            #greedy_action = greedy_policy(state, v_array)
            #if expected_value(state, greedy_action, v_array) > expected_value(state, new_policy[state], v_array):
            #    new_policy[state] = greedy_action

        if np.array_equal(new_policy, policy):
            stop = True
        else:
            policy = new_policy

    return policy

In [ ]:
#initial_policy = np.ones(len(states), dtype=int)
#initial_policy = np.zeros(len(states), dtype=int)
#initial_policy = np.array([0, 3, 3, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype='int') # Optimal policy found by Value Iteration

gamma = 0.99

#policy = policy_iteration(gamma=0.99, initial_policy=initial_policy)
policy = policy_iteration(gamma=gamma)

### Evaluate Policy Iteration with Gym (single trial)

In [ ]:
env._max_episode_steps = 1000

In [ ]:
reward_list = []

NUM_EPISODES = 1000

for episode_index in range(NUM_EPISODES):
    state = env.reset()
    done = False
    #t = 0

    while not done:
        action = policy[state]      # Take a random action
        state, reward, done, info = env.step(action)
        #t += 1

    reward_list.append(reward)
    #print("Episode finished after {} timesteps ; reward = {}".format(t, reward))

print(sum(reward_list) / NUM_EPISODES)            

env.close()

### Evaluate Policy Iteration for different $\gamma$ with confidence interval (bootstrap)

In [ ]:
%%time

NUM_EPISODES = 1000

reward_list = []

for gamma in (0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.84, 0.9, 0.95, 0.99):
    print("gamma:", gamma)
    policy = policy_iteration(gamma=gamma)
    
    for episode_index in range(NUM_EPISODES):
        state = env.reset()
        done = False

        while not done:
            action = policy[state]
            state, reward, done, info = env.step(action)

        reward_list.append({"gamma": gamma, "reward": reward})

env.close()

In [ ]:
df = pd.DataFrame(reward_list)
df.tail()

In [ ]:
# Plot mean reward (with its 95% confidence interval)

sns.relplot(x="gamma", y="reward", kind="line", data=df, height=6, aspect=1.5)
plt.axhline(0.76, color="red", linestyle=":", label="76% success threshold");   # 76% success threshold
plt.legend();

## Modified Policy Iteration (bonus)

The matrice inversion applied in `policy_evaluation` may be prohibtive for environments with big action space.

The *Modified Policy Iteration* use a simplified version of Value iteration to ensure the policy evaluation (simplified because there is no `max` operator anymore).

### Define Policy Evaluation functions

In [ ]:
def asynchronous_iterative_policy_evaluation(policy, gamma, num_iterations=30, display=False):
    """Estimate the value function `v_array` using an iterative methode (*modified policy iteration*)."""

    v_array = np.zeros(len(states))
    
    for iteration_index in range(num_iterations):
        
        for state in states:
            action = policy[state]
            v_array[state] = reward_array[state] + gamma * expected_value(state, action, v_array)
        
        if display:
            states_display(v_array, title="Value function", cbar=True)
            
    return v_array

In [ ]:
def iterative_policy_evaluation(policy, gamma, num_iterations=30, display=False):
    """Estimate the value function `v_array` using an iterative methode (*modified policy iteration*)."""

    new_v_array = np.zeros(len(states))
    v_array = np.zeros(len(states))
    
    for iteration_index in range(num_iterations):
        
        for state in states:
            action = policy[state]
            new_v_array[state] = reward_array[state] + gamma * expected_value(state, action, v_array)

        v_array = new_v_array

        if display:
            states_display(v_array, title="Value function", cbar=True)
    
    return v_array

### Compare Policy Evaluation functions

In [ ]:
policy = np.array([0, 3, 3, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype='int') # Optimal policy found by Value Iteration

display_policy(policy)

v_array = policy_evaluation(policy, gamma=gamma)
states_display(v_array, title="Value function", cbar=True)

v_array = iterative_policy_evaluation(policy, gamma=gamma, display=True)
states_display(v_array, title="Value function", cbar=True)

In [ ]:
#initial_policy = np.ones(len(states), dtype=int)
#initial_policy = np.zeros(len(states), dtype=int)
#initial_policy = np.array([0, 3, 3, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype='int') # Optimal policy found by Value Iteration

gamma = 0.99

#policy = policy_iteration(gamma=0.99, initial_policy=initial_policy)
policy = policy_iteration(gamma=gamma, policy_evaluation_function=iterative_policy_evaluation)

### Evaluate Modified Policy Iteration for different $\gamma$ with confidence interval (bootstrap)

In [ ]:
%%time

NUM_EPISODES = 2000

reward_list = []

for gamma in (0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.84, 0.9, 0.95, 0.99):
    print("gamma:", gamma)
    policy = policy_iteration(gamma=gamma, policy_evaluation_function=iterative_policy_evaluation)
    
    for episode_index in range(NUM_EPISODES):
        state = env.reset()
        done = False

        while not done:
            action = policy[state]
            state, reward, done, info = env.step(action)

        reward_list.append({"gamma": gamma, "reward": reward})

env.close()

In [ ]:
df = pd.DataFrame(reward_list)
df.tail()

In [ ]:
# Plot mean reward (with its 95% confidence interval)

sns.relplot(x="gamma", y="reward", kind="line", data=df, height=6, aspect=1.5)
plt.axhline(0.76, color="red", linestyle=":", label="76% success threshold");   # 76% success threshold
plt.legend();